In [ ]:
# see link
# https://github.com/Fraud-Detection-Handbook

package ='06-sklearn.ensemble'
name='RandomForest'
tuningAndParameters='06-NearMissV2 Undersampling'

hyperparametersFound={'max_depth': 4, 'n_estimators':227}

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import datamanagement as dm
reload(dm)

import result as resultMd
reload(resultMd)

import graph as gf
reload(gf)

print('done')

In [ ]:
dfLearning, dfValidation =dm.getDataLearningAndValidation()

dfLearning.head()

In [ ]:
#%%script false

from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import NearMiss

from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
from sklearn.metrics import f1_score

train_f1s=[]
test_f1s =[]
range= []

predictors = dm.getPredictors(dfLearning)
target = dm.getTarget()

x1, y1 = dfLearning[predictors], dfLearning[target]
sc = StandardScaler()
x2 = sc.fit_transform(x1)

TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0


x_train0, x_test, y_train0, y_test = train_test_split(x2, y1, test_size = TEST_SIZE, 
                                                        stratify=y1,
                                                        random_state = RANDOM_STATE)



neighbors = np.arange(1,11,1)
for n_neighbors in neighbors:
    print("n_neighbors ",n_neighbors)
    undersample = NearMiss(sampling_strategy=0.01,version=2, n_neighbors=n_neighbors)
    x3, y3 = undersample.fit_resample(x2,y1)
    x_train, y_train = undersample.fit_resample(x_train0, y_train0)

    modelClf = RandomForestClassifier(random_state=42)
    parameters=hyperparametersFound
    modelClf.set_params(**parameters)

    modelClf.fit(x_train, y_train)
    predsTrain = modelClf.predict(x_train)
    predsTest = modelClf.predict(x_test)

    train_f1=f1_score(y_train, predsTrain)
    print("f1 train {:.4f}".format(train_f1))
    
    test_f1=f1_score(y_test, predsTest)
    print("f1 test  {:.4f}".format(test_f1))
    
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(n_neighbors)
    print('-----------------------')

dm.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import NearMiss

TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0

predictors = dm.getPredictors(dfLearning)
target = dm.getTarget()

x1, y1 = dfLearning[predictors], dfLearning[target]
sc = StandardScaler()
x2 = sc.fit_transform(x1)




x_train0, x_test, y_train0, y_test = train_test_split(x2, y1, test_size = TEST_SIZE, 
                                                        stratify=y1,
                                                        random_state = RANDOM_STATE)


undersample = NearMiss(sampling_strategy=0.01,version=2, n_neighbors=5)
x3, y3 = undersample.fit_resample(x2,y1)
x_train, y_train = undersample.fit_resample(x_train0, y_train0)
print(x_train0.shape)
print(x_train.shape)

In [ ]:
%%script false

from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

dic_param={
    'n_estimators': randint(20,400),
    'max_depth': randint(2,12)
}
modelClf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)

#{'max_depth': 7, 'n_estimators': 349}
#0.7698865507639012


In [ ]:
%%script false

from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV

dic_param={   
    'n_estimators': [225,226,227,228,229,230],
    'max_depth': [3,4,5,6,7,8,9],
}
modelClf = RandomForestClassifier(random_state=42)

random_search = GridSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)

#{'max_depth': 7, 'n_estimators': 349}
#0.7698865507639012

#{'max_depth': 8, 'n_estimators': 300}
#0.7775387091813322

#{'max_depth': 4, 'n_estimators': 200}
#0.7315210497149166

#{'max_depth': 4, 'n_estimators': 225}
#0.7570095510791616

#{'max_depth': 4, 'n_estimators': 227}
#0.7577298806045557


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

modelClf = RandomForestClassifier(random_state=42)
parameters=hyperparametersFound

modelClf.set_params(**parameters)
then= datetime.now()
modelClf.fit(x_train, y_train)
now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result(package, name, tuningAndParameters, duration_in_s)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

F1Learning =f1_score(y_train, predsTrain)
F1Test=f1_score(y_test, predsTest)
dm.show_confusion_matrix(y_train, predsTrain,'Confusion matrix learning data')
print(f"f1 train {F1Learning:.3f}")
dm.show_confusion_matrix(y_test, predsTest,'Confusion matrix test data')
print(f"f1 test {F1Test:.3f}")
resultMd.update_learning_test_result(package, name, tuningAndParameters, F1Learning,F1Test)

In [ ]:
gf.show_importance(modelClf, predictors)

In [ ]:

gf.show_prediction_graph(modelClf, x_test,y_test)

In [ ]:
dfValidatationScaled = sc.transform(dfValidation[predictors])

predsValidation = modelClf.predict(dfValidatationScaled)
f1=f1_score(dfValidation[target], predsValidation)

dm.show_confusion_matrix(dfValidation[target], predsValidation,'Confusion matrix validation data')
print(f"f1 validation {f1:.3f}")
resultMd.update_performance_result(package, name, tuningAndParameters, f1)
resultMd.update_hyperparameters_result(package, name, tuningAndParameters, hyperparametersFound)